In [82]:
from mts.core.mtserie_dataset import MTSerieDataset
from mts.core.projections import ProjectionAlg
from models.emotion_dataset_controller import *
import matplotlib.pyplot as plt
from mts.core.projections import ProjectionAlg, euclidean_distance_matrix, mds_projection, compute_k_distance_matrixes, compute_distance_matrix
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed


config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
config.gpu_options.allow_growth = True
tf.keras.backend.set_session(tf.Session(config=config));


import sys
sys.path.insert(1, '/home/texs/Documents/AirQuality/repositories/peax/experiments')


In [83]:
controller = AppController()

In [84]:
# dataset_id = 'emotions_in_music'
# dataset_id = 'afew_va'
# dataset_id = 'aff-wild-categorical'
# dataset_id = "aff-wild-dimensional"
# dataset_id = "ascertain"
dataset_id = "drivers_stress"
controller.loadLocalDataset(dataset_id)

True

In [85]:

dataset = controller.datasets[dataset_id]
X = dataset.values()

print(X.shape)
print(X.max())
print(X.min())

(13, 11103, 1)
0.98516
0.0


In [86]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# class MinMaxScaler3D(MinMaxScaler):

#     def fit_transform(self, X, y=None):
#         x = np.reshape(X, newshape=(X.shape[0]*X.shape[1], X.shape[2]))
#         return np.reshape(super().fit_transform(x, y=y), newshape=X.shape)

# scaler = MinMaxScaler3D()
# X = scaler.fit_transform(X)
# print(X.shape)
# print(X.max())
# print(X.min())

In [87]:
from sklearn.preprocessing import MinMaxScaler

def normalizeWindow(data, percentile: float = 99.9):
    cutoff = np.percentile(data, (0, percentile))
    data_norm = np.copy(data)
    data_norm[np.where(data_norm < cutoff[0])] = cutoff[0]
    data_norm[np.where(data_norm > cutoff[1])] = cutoff[1]

    return MinMaxScaler().fit_transform(data_norm)

# X = dataset.values()
# X = np.array([  normalizeWindow(np.concatenate([x, np.zeros([1, 2])]) )    for x in X])
# X = np.array([(np.concatenate([x, np.zeros([1, 2])]) )    for x in X])


N, T, D =X.shape
# allWindows = np.split(X, 2, axis = 2)
# windows1 = allWindows[0]
# print(windows1.shape)
# print(windows2.shape)
print(X.shape)

(13, 11103, 1)


In [88]:
timesteps = X.shape[1]
n_features = X.shape[2]
# print(np.argwhere(np.isnan(X)))
# print(timesteps)

In [89]:
# define model
from keras import Model

# encodeLayer = LSTM(64, activation='relu', return_sequences=False)
# model = Sequential()
# model.add(LSTM(256, activation='relu', input_shape=(timesteps,n_features), return_sequences=True))
# model.add(LSTM(128, activation='relu', return_sequences=False))
# # model.add(encodeLayer)
# model.add(RepeatVector(timesteps))
# model.add(LSTM(128, activation='relu', return_sequences=True))
# model.add(LSTM(256, activation='relu', return_sequences=True))
# model.add(TimeDistributed(Dense(n_features)))
# model.compile(optimizer='adam', loss='mse')
# model.summary()

model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(timesteps,n_features), return_sequences=True))
model.add(LSTM(64, activation='relu', return_sequences=False))
model.add(RepeatVector(timesteps))
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(LSTM(128, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 11103, 128)        66560     
_________________________________________________________________
lstm_14 (LSTM)               (None, 64)                49408     
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 11103, 64)         0         
_________________________________________________________________
lstm_15 (LSTM)               (None, 11103, 64)         33024     
_________________________________________________________________
lstm_16 (LSTM)               (None, 11103, 128)        98816     
_________________________________________________________________
time_distributed_3 (TimeDist (None, 11103, 1)          129       
Total params: 247,937
Trainable params: 247,937
Non-trainable params: 0
________________________________________________

In [ ]:
model.fit(X, X, epochs=10, batch_size=50, verbose=1)
# demonstrate reconstruction
# yhat = model.predict(X, verbose=0)
# print('---Predicted---')
# print(np.round(yhat,3))
# print('---Actual---')
# print(np.round(X, 3))

Epoch 1/10
13/13 [==============================] - 29s 2s/step - loss: 0.4909
Epoch 2/10
13/13 [==============================] - 28s 2s/step - loss: 0.4751
Epoch 3/10
13/13 [==============================] - 28s 2s/step - loss: 0.4606
Epoch 4/10
13/13 [==============================] - 27s 2s/step - loss: 0.4430
Epoch 5/10
13/13 [==============================] - 27s 2s/step - loss: 0.4220
Epoch 6/10
13/13 [==============================] - 27s 2s/step - loss: 0.3949
Epoch 7/10
13/13 [==============================] - 27s 2s/step - loss: 0.3608
Epoch 8/10
13/13 [==============================] - 27s 2s/step - loss: 0.3177
Epoch 9/10
13/13 [==============================] - 27s 2s/step - loss: 0.2642
Epoch 10/10


In [ ]:
import matplotlib.pyplot as plt

index = 43
x1 = model.predict(np.array([X[index]]))[0]
y1 = X[index]


plt.plot(x1)

plt.show()

In [ ]:
plt.plot(y1)

plt.show()

In [ ]:
# encoder = Model(input=[model.layers[0]], output=[model.layers[2]])
inter_output_model = Model(model.input, model.get_layer(index = 1).output )
# inter_output_model.predict(np.array([X[10]])).shape
Y = inter_output_model.predict(X)


In [ ]:
# from sklearn.manifold import MDS

# print(Y.shape)
# mds = MDS(n_components=2)
# coords = mds.fit_transform(Y)



In [ ]:
import umap

reducer = umap.UMAP()
coords = reducer.fit_transform(Y)

In [ ]:
print(coords.shape)

plt.scatter(
    coords[:, 0], coords[:, 1], marker = 'o'
)
